In [1]:
# Fixing imports
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split as tts
from sklearn.datasets import fetch_openml
import time

In [2]:
x, y = fetch_openml('mnist_784', version=1, return_X_y=True)

In [3]:
# Fixing ys and xs for training
# x = x.reshape(len(x), 28, 28)

real_y = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] for i in range(len(y))])
for i in range(len(y)):
    real_y[i, int(y[i])] = 1.0

In [4]:
# Splitting to train and test sets
x_train, x_test, y_train, y_test = tts(x, real_y)
x_train = x_train.reshape([-1, 28, 28, 1])
x_test = x_test.reshape([-1, 28, 28, 1])

TRAINING_BATCHES = int(len(x_train)/100)
TESTING_BATCHES = int(len(x_test)/35)

x_train_batches = np.split(x_train, TRAINING_BATCHES)
x_test_batches = np.split(x_test, TESTING_BATCHES)
y_train_batches = np.split(y_train, TRAINING_BATCHES)
y_test_batches = np.split(y_test, TESTING_BATCHES)

In [5]:
class DigitRecognizer():
    def __init__(self):
        self.x = tf.placeholder(tf.float32, shape=[None, 28, 28, 1])
        self.y = tf.placeholder(tf.float32)
        
#         self.W1 = tf.Variable(tf.random_normal([5, 5, 1, 32]))
#         self.b1 = tf.Variable(tf.random_normal([32]))
#         self.W2 = tf.Variable(tf.random_normal([14*14*32, 10]))
#         self.b2 = tf.Variable(tf.random_normal([10]))
        
#         conv1 = tf.nn.bias_add(tf.nn.conv2d(self.x, self.W1, strides=[1, 1, 1, 1], padding='SAME'), self.b1)
#         pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        
        conv1 = tf.layers.conv2d(self.x, filters=32, kernel_size=[5, 5], strides=[1, 1], padding='SAME')
        pool1 = tf.layers.max_pooling2d(conv1, pool_size=[2, 2], strides=[2, 2], padding='SAME')
        conv2 = tf.layers.conv2d(pool1, filters=64, kernel_size=[5, 5], strides=[1, 1], padding='SAME')
        pool2 = tf.layers.conv2d(conv2, filters=64, kernel_size=[2, 2], strides=[2, 2], padding='SAME')
        
#         logits = tf.nn.bias_add(tf.matmul(tf.reshape(pool1, [-1, 14 * 14 * 32]), self.W2), self.b2)
    
        logits = tf.layers.dense(tf.layers.flatten(pool2), units=10)
        self.f = tf.nn.softmax(logits)
        self.accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(self.f, 1), tf.argmax(self.y, 1)), tf.float32))
        self.loss = tf.losses.softmax_cross_entropy(self.y, logits)

In [6]:
model = DigitRecognizer()
mini_op = tf.train.AdamOptimizer(0.001).minimize(model.loss)

session = tf.Session()
session.run(tf.global_variables_initializer())

start = time.time()

EPOCHS = 20

for epoch in range(EPOCHS):
    start_epoch = time.time()
    print("Epoch %d" % (epoch + 1))
    for batch in range(TRAINING_BATCHES):
        session.run(mini_op, {model.x: x_train_batches[batch], model.y: y_train_batches[batch]})
    end_epoch = time.time()
    accuracy = session.run([model.accuracy], {model.x: x_test, model.y: y_test})
    print("Model accuracy after %d epochs: %.2f" % (epoch, accuracy[0]))
    print("Time for epoch %.d: %.f\n" % (epoch, (end_epoch - start_epoch)))

end = time.time()
print("Runtime: %.2f" % (end - start))

# loss, accuracy = session.run([model.loss, model.accuracy], {model.x: x_test, model.y: y_test})

W0905 11:06:22.622452 139857906677568 deprecation.py:323] From <ipython-input-5-cb1a11fb4db3>:14: conv2d (from tensorflow.python.layers.convolutional) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
W0905 11:06:22.626780 139857906677568 deprecation.py:506] From /home/sindrtho/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0905 11:06:22.788205 139857906677568 deprecation.py:323] From <ipython-input-5-cb1a11fb4db3>:15: max_pooling2d (from tensorflow.python.layers.pooling) is deprecated and will be removed in a future version.
Instructions for updating:
Use keras.layers.MaxPooling2D instead.
W0905 11:06:22.897209 

Epoch 1


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv2d/Conv2D (defined at /tmp/tmpj4sk80as.py:12) ]]

Errors may have originated from an input operation.
Input Source operations connected to node conv2d/Conv2D:
 conv2d/kernel/read (defined at <ipython-input-5-cb1a11fb4db3>:14)	
 Placeholder (defined at <ipython-input-5-cb1a11fb4db3>:3)

Original stack trace for 'conv2d/Conv2D':
  File "/home/sindrtho/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/sindrtho/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/sindrtho/anaconda3/envs/tensorflow_gpuenv/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/sindrtho/anaconda3/envs/tensorflow_gpuenv/li

In [10]:
predictions, accuracy = session.run([model.f, model.accuracy], {model.x: x_test, model.y: y_test})
for p in predictions:
    max = 0
    index = 0
    for n in range(len(p)):
        if p[n] > max:
            max = p[n]
            index = n
    for n in range(len(p)):
        p[n] = 0 if not n == index else 1
        
acc = sum([int(list(predictions[i]) == list(y_test[i])) for i in range(len(predictions))])/len(predictions)*100
print("%.2f%s accuracy" % (acc, '%'))

94.03% accuracy


In [27]:
print(accuracy1)
print(accuracy2)

0.13491428
0.9489143


In [8]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 15097067270289232940, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 6933320332304797845
 physical_device_desc: "device: XLA_CPU device"]